# How can I redirect input and output of executed scripts

* **Difficulty level**: easy
* **Time need to lean**: 10 minutes or less
* **Key points**:
  * Option `input` and `output` redirect input and output

### `input`

Parameter `input` specifies the input files that an action needs before it can be executed. However, unlike targets in `input:` statement of a step where lacking an input target would trigger the execution of an auxiliary step (if needs) to produce it, SoS would yield an error if the input file does not exist.

For example, in the following example, step `20` is executed after step `10` so its `report` action can report the content of `a.txt` produced by step `10`.

In [10]:
%sandbox
%run
[10]
output: 'a.txt'
bash:
    echo 'content of a.txt' > a.txt

[20]
report: input='a.txt'

content of a.txt



However, in the following example, step `20` is executed as the first step of workflow `default`. The `report` action requires input file `a.txt` and yields an error.

In [11]:
%sandbox --expect-error
%run
[a: provides='a.txt']
bash:
    echo 'content of a.txt' > a.txt

[20]
report: input='a.txt'

ValueError: Input file a.txt does not exist.


`a.txt` has to be put into the input statement of step `20` for the auxiliary step to be executed:

In [12]:
%sandbox
%run
[a: provides='a.txt']
bash:
    echo 'content of a.txt' > a.txt

[20]
input: 'a.txt'
report: input=_input[0]

content of a.txt



Although all actions accept parameter `input` and SoS will always check the existence of specified input file, the action themselves might or might not make use of this parameter. Roughly speaking, script-executing actions such as `run`, `bash` and `python` accepts this parameter and prepend the content of all input files to the script; report-generation actions `report`, `pandoc` and `RMarkdown` append the content of input files after the specifie dscript, and other actions usually ignore this parameter.

For example, if you have a function that needs to be included in a Python script (more likely multiple scripts), you could define it in a separate file and include it with scripts defined in a `python` action: 

In [13]:
%run
# define a function and save to file myfunc.inc
report: output="myfunc.inc"
  def myfunc():
    print("Hello")

[1]
python: input='myfunc.inc'
    myfunc()

Hello


### `output`

Similar to `input`, parameter `output` defines the output of an action, which can be a single name (or target) or a list of files or targets. SoS would check the existence of output target after the completion of the action. For example, 

In [14]:
%sandbox --expect-error
%run
[10]
bash: output='non_existing.txt'

RuntimeError: Output target non_existing.txt does not exist after completion of action bash


In addition to checking the existence of input and output files, specifying `input` and `output` of an action will allow SoS to create signatures of action so that it will not be executed when it is called again with the same input and output files. This is in addition to step-level signature and can be useful for long-running actions.

For example, suppose action `sh` is time-consuming that produces output `test.txt`

In [15]:
%run -s default
[10]
import time, os
time.sleep(2)

sh: input=[], output='test.txt'
   touch test.txt

print(os.path.getmtime('test.txt'))


1512781007.0


Because the action has parameter `input` and `output`, a signature will be created so it will not be re-executed even when the step itself is changed (from `sleep(2)` to `sleep(1)`).

In [16]:
%run -s default
[10]
import time, os
time.sleep(1)

sh: input=[], output='test.txt'
   touch test.txt

print(os.path.getmtime('test.txt'))


1512781008.0


Note that we have to use option `-s default` for our examples because the default mode for SoS in Jupyter is `ignore` so no siguatures will be saved and used by default.

### `stdout`

Option `stdout` is applicable to script-executing actions such as `bash` and `R` and redirect the standard out of the action to specified file. The value of the option should be a path-like object (`str`, `path`, etc), or `False`. The file will be opened in `append` mode so you will have to remove or truncate the file if the file already exists. If `stdout=False`, the output will be suppressed (redirect to `/dev/null` under linux).

For example,

In [11]:
!rm -f test.log

sh: stdout='test.log'
ls *.ipynb

In [12]:
!head -2 test.log

Extending_SoS.ipynb
Language_Module.ipynb


### `stderr`

Option `stderr` is similar to `stdout` but redirects the standard error output of actions. `stderr=False` also suppresses stderr.

## Further reading

* 